# kurulum

In [17]:
!pip install openai


[notice] A new release of pip available: 22.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
!pip show openai

Name: openai
Version: 1.97.0
Summary: The official Python library for the openai API
Home-page: 
Author: 
Author-email: OpenAI <support@openai.com>
License: Apache-2.0
Location: c:\users\cagin\appdata\local\programs\python\python310\lib\site-packages
Requires: anyio, distro, httpx, jiter, pydantic, sniffio, tqdm, typing-extensions
Required-by: 


In [19]:
import sys
print(sys.executable)

C:\Users\cagin\AppData\Local\Programs\Python\Python313\python.exe


In [20]:
!C:\Users\cagin\AppData\Local\Programs\Python\Python313\python.exe -m pip install openai


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\cagin\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip


# kod

In [21]:
## key: sk-7427782d5a614814b8f422dcc6b0ffbd
## key: sk-7427782d5a614814b8f422dcc6b0ffbd
## key: sk-7427782d5a614814b8f422dcc6b0ffbd

In [36]:
from datetime import datetime, time, timezone

def is_off_peak():
    # Şu anki UTC saatini timezone bilgisiyle birlikte alıyoruz
    utc_now = datetime.now(timezone.utc).time()
    
    # Off-peak başlangıç ve bitiş saatleri (UTC)
    start_off_peak = time(16, 30)  # 16:30 UTC - 19:30 TR
    end_off_peak = time(0, 30)     # 00:30 UTC - 03:30 TR (geceyarısından sonraki saat)
    
    if start_off_peak < end_off_peak:
        # Normal durum (aynı gün içinde)
        return start_off_peak <= utc_now <= end_off_peak
    else:
        # Gece yarısını aşan durum (örnek: 16:30 - 00:30)
        return utc_now >= start_off_peak or utc_now <= end_off_peak

def print_cost_info():
    if is_off_peak():
        print("💰 Şu an off-peak saatindesiniz (19:30 - 03:30). DeepSeek-R1 sorgu başına %75 indirimle $0.0002 harcanır.")
    else:
        print("💰 Normal saatlerde DeepSeek-R1 sorgu başına $0.0008 ücretlendirilirsiniz.")

# Kullanımı:
print_cost_info()

💰 Şu an off-peak saatindesiniz (19:30 - 03:30). DeepSeek-R1 sorgu başına %75 indirimle $0.0002 harcanır.


In [29]:
import requests

api_key = "sk-7427782d5a614814b8f422dcc6b0ffbd"
base_url = "https://api.deepseek.com/v1/chat/completions"

headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

chat_history = [{"role": "system", "content": "You are a helpful assistant."}]

print("💬 DeepSeek-R1 Chatbox'a hoş geldiniz!")
print("⌛ Türkiye saatiyle 19:30 - 03:30 arası ücretsizdir.")
print("🚪 Çıkmak için 'exit' yazın.\n")

while True:
    user_input = input("👤 Sen: ")
    if user_input.lower() == "exit":
        print("👋 Görüşmek üzere!")
        break

    chat_history.append({"role": "user", "content": user_input})

    data = {
        "model": "deepseek-reasoner",
        "messages": chat_history
    }

    response = requests.post(base_url, json=data, headers=headers)

    if response.status_code == 200:
        assistant_reply = response.json()['choices'][0]['message']['content']
        chat_history.append({"role": "assistant", "content": assistant_reply})
        print("🤖 DeepSeek:", assistant_reply)
    else:
        print(f"❌ Hata {response.status_code}: {response.text}")

💬 DeepSeek-R1 Chatbox'a hoş geldiniz!
⌛ Türkiye saatiyle 19:30 - 03:30 arası ücretsizdir.
🚪 Çıkmak için 'exit' yazın.



👤 Sen:  merhaba yardıma ihtiyacım var


🤖 DeepSeek: Merhaba! Tabii ki, size yardımcı olmak için buradayım. 🧡  
Lütfen ihtiyacınız olan konuyu belirtin (örneğin: ders, teknik sorun, günlük hayat, planlama, vb.). Ne kadar detay verirseniz o kadar iyi destek olabilirim.  

Örnekler:  
- **"Matematik sorusu çözemiyorum…"**  
- **"Bilgisayarım açılmıyor, ne yapmalıyım?"**  
- **"İngilizce öğrenmek istiyorum, nasıl başlamalıyım?"**  

Hemen sorunuzu yazın, birlikte çözüm bulalım! ✨


👤 Sen:  exit


👋 Görüşmek üzere!


# fatura yazdırma

In [47]:
def generate_invoice(text: str) -> str:
    # Geliştirilmiş desenler
    name_pattern = r"(?:adı\s*soyadı\s*:)?\s*(?:biraz önce\s*)?([A-ZÇĞİÖŞÜ][a-zçğıöşü]+\s+[A-ZÇĞİÖŞÜ][a-zçğıöşü]+)"
    date_pattern = r"(?:fatura\s*(?:kesim)?\s*tarihi\s*:)?\s*([0-9]{1,2}\s*\w+\s*[0-9]{4})"
    amount_pattern = r"([0-9]{1,3}(?:[.,][0-9]{3})*(?:[.,][0-9]{2})?)\s*(tl|₺|dolar|usd)?"
    desc_pattern = r"(?:açıklama|detay|nedeni|sebep)\s*:\s*([^,]+)|([a-zçğıöşü\s]+ücreti)"

    # Eşleştirme
    name = re.search(name_pattern, text, re.IGNORECASE)
    date = re.search(date_pattern, text, re.IGNORECASE)
    amount_match = re.search(amount_pattern, text, re.IGNORECASE)
    description = re.search(desc_pattern, text, re.IGNORECASE)

    # Verileri ayrıştır
    name = name.group(1).strip() if name else "Belirtilmedi"
    date = date.group(1).strip() if date else datetime.now().strftime("%d %B %Y")
    
    if amount_match:
        amount = amount_match.group(1).replace(",", ".").strip()
        currency = amount_match.group(2) or "tl"
        currency_symbol = "₺" if "tl" in currency.lower() or "₺" in currency else "$"
    else:
        amount = "0.00"
        currency_symbol = "$"

    if description:
        description = (description.group(1) or description.group(2)).strip()
    else:
        description = "Yapay zekâ kullanım hizmeti"

    # Fatura metni
    invoice = f"""📄 FATURA

👤 Adı Soyadı: {name}
🗓️ Fatura Tarihi: {date}
💵 Tutar: {currency_symbol}{amount}
📝 Açıklama: {description}

Teşekkür ederiz. Bu fatura, {date} tarihinde yapay zekâ destekli hizmet kullanımı karşılığında kesilmiştir.
"""
    return invoice

In [48]:
def start_invoice_chat():
    print("🧾 Yapay Zekâ Destekli Fatura Chatbotuna Hoş Geldiniz.")
    print("Lütfen aşağıdaki gibi bir mesaj girin:\n")
    print("Örnek: adı soyadı: Ahmet Yılmaz, fatura: 24 Temmuz 2025, tutar: 350 dolar, açıklama: danışmanlık ücreti\n")
    print("❌ Çıkmak için: 'çık', 'exit', 'quit' yazabilirsiniz.\n")

    while True:
        user_input = input("🗨️ Siz: ")
        if user_input.lower() in ['çık', 'exit', 'quit']:
            print("🔚 Chat sonlandırıldı.")
            break

        invoice = generate_invoice(user_input)
        print("\n📤 Oluşturulan Fatura:\n")
        print(invoice)
        print("━━━━━━━━━━━━━━━━━━━━━━━━━━━\n")

In [49]:
start_invoice_chat()

🧾 Yapay Zekâ Destekli Fatura Chatbotuna Hoş Geldiniz.
Lütfen aşağıdaki gibi bir mesaj girin:

Örnek: adı soyadı: Ahmet Yılmaz, fatura: 24 Temmuz 2025, tutar: 350 dolar, açıklama: danışmanlık ücreti

❌ Çıkmak için: 'çık', 'exit', 'quit' yazabilirsiniz.



🗨️ Siz:  biraz önce Çağay Çatal 400 tl şirket içi abolnelik aldı



📤 Oluşturulan Fatura:

📄 FATURA

👤 Adı Soyadı: Çağay Çatal
🗓️ Fatura Tarihi: 24 July 2025
💵 Tutar: ₺400
📝 Açıklama: Yapay zekâ kullanım hizmeti

Teşekkür ederiz. Bu fatura, 24 July 2025 tarihinde yapay zekâ destekli hizmet kullanımı karşılığında kesilmiştir.

━━━━━━━━━━━━━━━━━━━━━━━━━━━



🗨️ Siz:  exit


🔚 Chat sonlandırıldı.


# deepseek + fatura

In [ ]:
import re
import requests
from datetime import datetime

api_key = "sk-7427782d5a614814b8f422dcc6b0ffbd"
base_url = "https://api.deepseek.com/v1/chat/completions"

headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

chat_history = [{"role": "system", "content": "You are a helpful assistant that extracts invoice data (name, date, amount, description) from user input."}]

def generate_invoice(text: str) -> str:
    # Geliştirilmiş desenler
    name_pattern = r"(?:adı\s*soyadı\s*:)?\s*(?:biraz önce\s*)?([A-ZÇĞİÖŞÜ][a-zçğıöşü]+\s+[A-ZÇĞİÖŞÜ][a-zçğıöşü]+)"
    date_pattern = r"(?:fatura\s*(?:kesim)?\s*tarihi\s*:)?\s*([0-9]{1,2}\s*\w+\s*[0-9]{4})"
    amount_pattern = r"([0-9]{1,3}(?:[.,][0-9]{3})*(?:[.,][0-9]{2})?)\s*(tl|₺|dolar|usd)?"
    desc_pattern = r"(?:açıklama|detay|nedeni|sebep)\s*:\s*([^,]+)|([a-zçğıöşü\s]+ücreti)"

    # Eşleştirme
    name = re.search(name_pattern, text, re.IGNORECASE)
    date = re.search(date_pattern, text, re.IGNORECASE)
    amount_match = re.search(amount_pattern, text, re.IGNORECASE)
    description = re.search(desc_pattern, text, re.IGNORECASE)

    # Verileri ayrıştır
    name = name.group(1).strip() if name else "Belirtilmedi"
    date = date.group(1).strip() if date else datetime.now().strftime("%d %B %Y")
    
    if amount_match:
        amount = amount_match.group(1).replace(",", ".").strip()
        currency = amount_match.group(2) or "tl"
        currency_symbol = "₺" if "tl" in currency.lower() or "₺" in currency else "$"
    else:
        amount = "0.00"
        currency_symbol = "$"

    if description:
        description = (description.group(1) or description.group(2)).strip()
    else:
        description = "Yapay zekâ kullanım hizmeti"

    # Fatura metni
    invoice = f"""📄 FATURA

👤 Adı Soyadı: {name}
🗓️ Fatura Tarihi: {date}
💵 Tutar: {currency_symbol}{amount}
📝 Açıklama: {description}

Teşekkür ederiz. Bu fatura, {date} tarihinde yapay zekâ destekli hizmet kullanımı karşılığında kesilmiştir.
"""
    return invoice

def start_invoice_chat():
    print("🧾 Yapay Zekâ Destekli Fatura Chatbotuna Hoş Geldiniz.")
    print("Lütfen fatura bilgilerinizi doğal dil ile yazınız.")
    print("❌ Çıkmak için: 'çık', 'exit', 'quit' yazabilirsiniz.\n")

    while True:
        user_input = input("🗨️ Siz: ")
        if user_input.lower() in ['çık', 'exit', 'quit']:
            print("🔚 Chat sonlandırıldı.")
            break

        # DeepSeek API'ye gönder
        chat_history.append({"role": "user", "content": user_input})
        data = {
            "model": "deepseek-reasoner",
            "messages": chat_history
        }

        response = requests.post(base_url, json=data, headers=headers)
        if response.status_code == 200:
            assistant_reply = response.json()['choices'][0]['message']['content']
            chat_history.append({"role": "assistant", "content": assistant_reply})
            print("🤖 DeepSeek:", assistant_reply)

            # DeepSeek'ten gelen cevabı kullanarak fatura oluştur
            invoice_text = generate_invoice(assistant_reply)
            print("\n📤 Oluşturulan Fatura:\n")
            print(invoice_text)
            print("━━━━━━━━━━━━━━━━━━━━━━━━━━━\n")

        else:
            print(f"❌ Hata {response.status_code}: {response.text}")

start_invoice_chat()

🧾 Yapay Zekâ Destekli Fatura Chatbotuna Hoş Geldiniz.
Lütfen fatura bilgilerinizi doğal dil ile yazınız.
❌ Çıkmak için: 'çık', 'exit', 'quit' yazabilirsiniz.



🗨️ Siz:  selam, biraz önce şirketimize çağın çatal adındaki birisi geldi ve 545 liralık danışmanlık ücreti aldı


🤖 DeepSeek: Aşağıda, verdiğiniz bilgilerden çıkarılan fatura verileri bulunmaktadır:

```json
{
  "name": "çağın çatal",
  "date": "2023-10-09",  // (Tarih belirtilmediği için işlem günü varsayılmıştır)
  "amount": 545.00,
  "currency": "TRY",
  "description": "danışmanlık ücreti"
}
```

### Açıklamalar:
- **İsim**: "çağın çatal" olarak çıkarıldı (metinde geçtiği şekliyle).
- **Tarih**: Girdide açık tarih belirtilmediği için, işlemin bugün (9 Ekim 2023) yapıldığı varsayılmıştır. Kesin tarih için ek bilgi gereklidir.
- **Tutar**: "545 lira" olarak belirtilen miktar, `545.00 TRY` şeklinde sayısallaştırıldı.
- **Açıklama**: "danışmanlık ücreti" hizmet detayı olarak kaydedildi.

Eksik veya düzeltilmesi gereken bilgiler varsa lütfen bildirin!

📤 Oluşturulan Fatura:

📄 FATURA

👤 Adı Soyadı: verdiğiniz bilgilerden
🗓️ Fatura Tarihi: 9 Ekim 2023
💵 Tutar: ₺202
📝 Açıklama: danışmanlık ücreti

Teşekkür ederiz. Bu fatura, 9 Ekim 2023 tarihinde yapay zekâ destekli hizmet kullanımı karşılığında kesil